In [1]:
import numpy as np
import matplotlib.pyplot as pl
import time
import pyfftw

In [2]:
from varname import nameof, argname
def copier(arg):
    og_name = argname('arg')
    new_name = og_name + '_copy'
    globals()[new_name] = arg

In [3]:
c = 3e8
G = 6.67e-11
pi = np.pi
const = 96/5*pi**(8/3)*(G/c**3)**(5/3)

Specific arguments

In [4]:
f0 = 120
Mc = 1e-4* 2e30
f_max = 200
T_obs = 1e4
beta = const*f0**(8/3)*Mc**(5/3)

f_signal = 40*f_max
nt = round(f_signal*T_obs)
t = np.arange(nt)/f_signal

f_ratio = 25
f_new = f_signal/f_ratio

In [5]:
beta

4.3971470309528426e-08

Generating signal and adding noise

In [6]:
phi = -6*pi/5*f0*(1-8./3.*beta*t)**(5/8)/beta
phi = np.mod(phi,2*pi)
signal = 1*np.exp(1j*phi)

nh = 50
noise = nh*np.random.normal(size = nt)
data = signal + noise

In [7]:
%reset_selective -f "^signal$"
%reset_selective -f noise
%reset_selective -f phi

In [8]:
%load_ext cython

In [41]:
%%cython --f
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp
cimport numpy as np
import numpy as np
import cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)

cdef int nogil_round(double a) nogil:
    return int(a)

def cython_minimal(float beta,
                 double[:] t,
                 complex[:] data,
                 float f_new):
    assert t.shape[0] == data.shape[0]
    cdef Py_ssize_t i
    cdef long[:] new_t = np.zeros((t.shape[0]), dtype = np.int64)
    cdef long[:] new_t_view = new_t
    
    for i in prange(t.shape[0], nogil=True):
        new_t_view[i] = nogil_round(f_new*-3/5*(1-8/3*beta*t[i])**(5/8)/beta)
    return new_t

In file included from /home/neil/anaconda3/envs/PBHs/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /home/neil/anaconda3/envs/PBHs/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/neil/anaconda3/envs/PBHs/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/neil/.cache/ipython/cython/_cython_magic_9ec7c594cd7908085851771f01468a38.c:735:
/home/neil/anaconda3/envs/PBHs/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [44]:
test = f_new*-3/5*(1-8/3*beta*t)**(5/8)/beta

In [47]:
type(test[0])

numpy.float64

In [ ]:
np.nonzero(np.diff(test))

In [42]:
np.array(cython_minimal(beta, t, data, f_new))

array([-2147483648, -2147483648, -2147483648, ..., -2147483648,
       -2147483648, -2147483648])

In [19]:
def strobo_wrapper(beta, t, data, f_new):
    new_t = cython_minimal(beta, t, data, f_new)
    idx = np.nonzero(np.diff(new_t)) #The step that downsamples
    
    resampled = data[idx]
    t_out = (new_t[idx]-new_t[0])/f_new
    return resampled, t_out

In [20]:
np.nonzero(np.diff(cython_minimal(beta, t, data, f_new)))

(array([], dtype=int64),)

In [11]:
out, t_out = strobo_wrapper(beta, t, data, f_new)
corrected = np.fft.fftshift(np.fft.fft(out))
freq_corrected = np.fft.fftshift(np.fft.fftfreq(len(t_out), d=t_out[1]-t_out[0]))
corrected = corrected[len(corrected)//2:]
freq_corrected = freq_corrected[len(freq_corrected)//2:]
nt_new = len(out)

ValueError: Invalid number of FFT data points (0) specified.

In [ ]:
pl.plot(freq_corrected, np.abs(corrected/nt_new))

## Numerically estimating error

In [ ]:
pyfftw.interfaces.cache.enable()
wisdom = pyfftw.export_wisdom()
pyfftw.import_wisdom(wisdom)

In [ ]:
def height_calc(beta):
    out, t_out = strobo_wrapper(beta, t, data, f_new)
    temp_in = pyfftw.empty_aligned(len(out), dtype='complex128')
    temp_out = pyfftw.empty_aligned(len(out), dtype='complex128')
    fft_obj = pyfftw.builders.fft(out)
    corrected = np.fft.fftshift(fft_obj())
    freq_corrected = np.fft.fftshift(np.fft.fftfreq(len(t_out), d=t_out[1]-t_out[0]))
    corrected = corrected[len(corrected)//2:]
    freq_corrected = freq_corrected[len(freq_corrected)//2:]
    nt_new = len(out)
    resampled_amplitudes = np.abs(corrected/nt_new)
    
    arg_max = np.argmax(resampled_amplitudes)
    peak_freq = freq_corrected[arg_max]
    peak_height = resampled_amplitudes[arg_max]
    return peak_freq, peak_height

In [ ]:
tic = time.time()

offset_arr = np.logspace(-6, -3, 100)
ref_freq, ref_height = height_calc(beta)

offset_results = np.array([height_calc(beta+i*beta) for i in offset_arr])
toc = time.time()
print((toc-tic)/60)

In [ ]:
# pl.plot(offset_arr, offset_results[:,0]/ref_freq)
# pl.show()
pl.semilogx(offset_arr, offset_results[:,1]/ref_height)
pl.xlabel(r'$\Delta \beta / \beta$')
pl.ylabel('Height ratio')
pl.title(r'$\beta_{exact} = %.1E$' % beta)
pl.show()

In [68]:
f0 = 120
Mc = 1e-4 * 2e30
f_max = 200
T_obs = 1e4
beta = const*f0**(8/3)*Mc**(5/3)

f_signal = 40*f_max
nt = round(f_signal*T_obs)
t = np.arange(nt)/f_signal

f_ratio = 25
f_new = f_signal/f_ratio

phi = -6*pi/5*f0*(1-8./3.*beta*t)**(5/8)/beta
phi = np.mod(phi,2*pi)
signal = 1*np.exp(1j*phi)

nh = 50
noise = nh*np.random.normal(size = nt)
data = signal + noise

In [69]:
tic = time.time()

offset_arr = np.logspace(-6, -3, 100)
ref_freq, ref_height = height_calc(beta)

offset_results = np.array([height_calc(beta+i*beta) for i in offset_arr])
toc = time.time()
print((toc-tic)/60)

0


ValueError: Zero length array: The input array should have no zero lengthaxes over which the FFT is to be taken

In [ ]:
# pl.plot(offset_arr, offset_results[:,0]/ref_freq)
# pl.show()
pl.semilogx(offset_arr, offset_results[:,1]/ref_height)
pl.xlabel(r'$\Delta \beta / \beta$')
pl.ylabel('Height ratio')
pl.title(r'$\beta_{exact} = %.1E$' % beta)
pl.show()

In [ ]:
tic = time.time()
beta = 1e-7
offset_arr = np.logspace(-8, -3, 100)
ref_freq, ref_height = height_calc(beta)

offset_results = np.array([height_calc(beta+i*beta) for i in offset_arr])
toc = time.time()
print((toc-tic)/60)

In [ ]:
# pl.plot(offset_arr, offset_results[:,0]/ref_freq)
# pl.show()
pl.semilogx(offset_arr, offset_results[:,1]/ref_height)
pl.xlabel(r'$\Delta \beta / \beta$')
pl.ylabel('Height ratio')
pl.title(r'$\beta_{exact} = %.1E$' % beta)
pl.show()

In [ ]:
tic = time.time()
beta = 1e-3
offset_arr = np.logspace(-8, -3, 100)
ref_freq, ref_height = height_calc(beta)

offset_results = np.array([height_calc(beta+i*beta) for i in offset_arr])
toc = time.time()
print((toc-tic)/60)

In [ ]:
# pl.plot(offset_arr, offset_results[:,0]/ref_freq)
# pl.show()
pl.semilogx(offset_arr, offset_results[:,1]/ref_height)
pl.xlabel(r'$\Delta \beta / \beta$')
pl.ylabel('Height ratio')
pl.title(r'$\beta_{exact} = %.1E$' % beta)
pl.show()